<a href="https://colab.research.google.com/github/anuraagbarde/market-strategies/blob/master/public_nifty_intraday_strategies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing

In [23]:
# Import the csv
import pandas as pd
df = pd.read_csv('NIFTY_2015.csv')
# print(df)

In [24]:
# Preprocessing dataframe
def convertStringDateToDateObject(nDate: int):
    sDate = str(nDate)
    year = int(sDate[0:4])
    month = int(sDate[4:6])
    date = int(sDate[6:])
    res = {
        "year": year,
        "month": month,
        "date": date
    }
    return res

def extractYear(sDate):
    return convertStringDateToDateObject(sDate)['year']

def extractMonth(sDate):
    return convertStringDateToDateObject(sDate)['month']

def extractDate(sDate):
    return convertStringDateToDateObject(sDate)['date']

df['year'] = df.apply(lambda row: extractYear(row.Date), axis=1)
df['month'] = df.apply(lambda row: extractMonth(row.Date), axis=1)
df['date'] = df.apply(lambda row: extractDate(row.Date), axis=1)

df.drop(['Date', 'Instrument'], axis = 1, inplace = True)

df_global = df.copy()
print(df_global)

        Time     Open     High      Low    Close  year  month  date
0      09:16  8276.00  8276.70  8252.70  8255.00  2015      1     1
1      09:17  8252.45  8255.00  8251.70  8254.10  2015      1     1
2      09:18  8253.70  8261.40  8253.70  8261.40  2015      1     1
3      09:19  8261.15  8264.55  8259.35  8259.60  2015      1     1
4      09:20  8259.10  8259.45  8253.05  8253.80  2015      1     1
...      ...      ...      ...      ...      ...   ...    ...   ...
92619  15:26  7946.70  7948.45  7945.85  7947.05  2015     12    31
92620  15:27  7946.65  7948.00  7942.60  7942.60  2015     12    31
92621  15:28  7943.85  7944.10  7941.55  7942.60  2015     12    31
92622  15:29  7942.40  7947.00  7941.05  7942.30  2015     12    31
92623  15:30  7943.10  7945.55  7939.50  7939.75  2015     12    31

[92624 rows x 8 columns]


Use `df = df_global.copy()` at the start of each new strategy

# STRATEGY 1

In [29]:
# Percentages
PROFIT_CUTOFF = 1
LOSS_CUTOFF = -3.00

# ZERODHA
TRANSACTION_COST_BUY = (58.37/100000)*100
TRANSACTION_COST_SELL = (168.38/100000)*100
TRANSACTION_COST = TRANSACTION_COST_BUY + TRANSACTION_COST_SELL

# TAX
# Ideally it should be filled under ITR-3 as a business income with abilty
# to show transaction cost as expense, but that form is hard to fill, also
# you can switch to old/new(section 115BAC) regime only once in lifetime once
# you fill ITR-3

# Anyways, lets say you show it as STCG(Short Term Captial Gain) under
# applicable tax slab in ITR-2 with the applicable tax slab being 30%.
# Also losses can be carry forwared for 4 years
TAX_SLAB = 30

# Typical for economically growing countries like India
INFLATION = 6

In [30]:
# Strategy functions

def calculatePercentage(start: float, curr: float)->float:
    # print('movement -> ', (curr-start)*100/start)
    return (curr-start)*100/start

def isProfited(start: float, curr: float)->bool:
    if(calculatePercentage(start, curr) > PROFIT_CUTOFF):
        print("Boom!")
        return True
    else:
        return False

def isLost(start: float, curr: float)->bool:
    if(calculatePercentage(start, curr) < LOSS_CUTOFF):
        print("Noo!")
        return True
    else:
        return False

class Day:
    def __init__(self, _date, _start):
        self.date = _date
        self.start = _start
        self.activity = False

    def setProfit(self):
        self.activity = True
        self.profit = True
        self.loss = False

    def setLoss(self):
        self.activity = True
        self.profit = False
        self.loss = True

    def display(self):
        print("Date", self.date)
        print("activity", self.activity)
        if(self.activity == True):
            print("profit", self.profit)
            print("loss", self.loss)
        print()


In [34]:
# Execute strategy

STATS_inactive = 0
STATS_profit = 0
STATS_loss = 0

# BOILERPLATE-START

YEAR = 2015
MONTHs = range(1, 12+1)

for mm in MONTHs:
    print("Month", mm)
    df = df_global.copy()
    df = df.where((df['year']== YEAR) & (df['month']== mm))
    df = df.dropna()
    DATEs = list(map(lambda x: int(x), df.date.unique()))
    for dd in DATEs:
        df_today = df.copy()
        df_today = df.where(df['date'] == dd)
        df_today = df_today.dropna()
        # BOILERPLATE-END

        # STRATEGY-START
        today = Day(dd, df_today['Open'].iloc[0])

        for _, row in df_today.iterrows():
            if(isProfited(today.start, float(row['High']))):
                today.setProfit()
                break

            if(isLost(today.start, float(row["Low"]))):
                today.setLoss()
                break

        if(today.activity == True):
            if(today.loss == True):
                STATS_loss +=1
            if(today.profit == True):
                STATS_profit +=1
        elif(today.activity == False):
            STATS_inactive += 1
        # today.display()

        # STRATEGY-END

print("STATS_inactive", STATS_inactive)
print("STATS_profit", STATS_profit)
print("STATS_loss", STATS_loss)
print("STATS_transaction_cost", TRANSACTION_COST*(STATS_profit + STATS_loss))
profit = STATS_profit*PROFIT_CUTOFF + STATS_loss*LOSS_CUTOFF - TRANSACTION_COST*(STATS_profit + STATS_loss)
print("profit after transaction cost:", profit)
profit_after_inflation = profit - (INFLATION)
print("profit after inflation:", profit_after_inflation, "at", INFLATION, "%")
profit_after_tax = profit_after_inflation*(1-(TAX_SLAB/100))
print("profit after tax:", profit_after_tax, "at", TAX_SLAB, "%")

Month 1
Boom!
Boom!
Boom!
Month 2
Boom!
Boom!
Boom!
Month 3
Boom!
Boom!
Month 4
Boom!
Boom!
Month 5
Boom!
Boom!
Boom!
Boom!
Boom!
Month 6
Boom!
Boom!
Boom!
Month 7
Boom!
Boom!
Boom!
Boom!
Month 8
Boom!
Noo!
Noo!
Boom!
Month 9
Boom!
Boom!
Boom!
Boom!
Boom!
Month 10
Boom!
Month 11
Boom!
Boom!
Month 12
Boom!
Boom!
STATS_inactive 211
STATS_profit 34
STATS_loss 2
STATS_transaction_cost 8.163
profit after transaction cost: 19.837
profit after inflation: 13.837 at 6 %
profit after tax: 9.685899999999998 at 30 %
